In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np

/Users/marcosgallo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/marcosgallo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/marcosgallo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/marcosgallo/anaconda3/lib/python3.7/site-packages/tensorf

In [15]:
# Load files
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
data = load_doc('data/shakespeare.txt')

#Tokenize
corpus = data.lower().split("\n")
tokenizer = Tokenizer(filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(corpus)
#Find total number of words for the loop
total_words = len(tokenizer.word_index) + 1

In [17]:
input_sequences = []
# Transform text into list of numbers...
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
# Create n-grams
for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)
# add zeros to the beginnings of sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# Create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
# One-hot encoding
label = ku.to_categorical(label, num_classes=total_words)

In [18]:
# LSTM model
model = Sequential()
# Adding word embeddings here with 100 dimensions
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
# Bidirectional LSTM to keep information from both past and future...
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# Final layer, size = number of words in corpus
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 100)            338600    
_________________________________________________________________
bidirectional (Bidirectional (None, 7, 300)            301200    
_________________________________________________________________
dropout (Dropout)            (None, 7, 300)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1693)              170993    
_________________________________________________________________
dense_1 (Dense)      

In [ ]:
history = model.fit(predictors, label, epochs=200, verbose=1)

In [ ]:
model.save('model-embeddings.h5')
dump(mapping, open('mapping-embeddings.pkl', 'wb'))

In [ ]:
# load the model
model = load_model('model.h5')

seed_text = "o know sweet love"
next_words = 50
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)